This notebook converts the nanogpt weights to the Transformer Lens format and verifies that the conversion was done correctly. I'm keeping it around as a reference. The script in `model_setup.py` does the same thing.

Step One: This first cell is model.py from NanoGPT copypasted into a cell for easy Colab use. I made one change: from:

```logits = self.lm_head(x[:, [-1], :])```

to:

```logits = self.lm_head(x[:, :, :])```

So I can compare the logits generated at all positions, rather than just the predicted next token.

In [ ]:
"""
Full definition of a GPT Language Model, all of it in this single file.
References:
1) the official GPT-2 TensorFlow implementation released by OpenAI:
https://github.com/openai/gpt-2/blob/master/src/model.py
2) huggingface/transformers PyTorch implementation:
https://github.com/huggingface/transformers/blob/main/src/transformers/models/gpt2/modeling_gpt2.py
"""

import math
import inspect
from dataclasses import dataclass

import torch
import torch.nn as nn
from torch.nn import functional as F

import math
import inspect
from dataclasses import dataclass

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.nn.init as init

class LayerNorm(nn.Module):
    """ LayerNorm but with an optional bias. PyTorch doesn't support simply bias=False """

    def __init__(self, ndim, bias=True):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None

    def forward(self, input):
        return F.layer_norm(input, self.weight.shape, self.weight, self.bias, 1e-5)

class CausalSelfAttention(nn.Module):
    """Masked MSSA Attention"""

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        # self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        self.c_attn = nn.Linear(config.n_embd, config.n_embd, bias=False)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=True)
        # regularization
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.dropout = config.dropout
        # flash attention make GPU go brrrrr but support is only in PyTorch >= 2.0
        # self.flash = hasattr(torch.nn.functional, 'scaled_dot_product_attention')
        self.flash = False
        if not self.flash:
            print("WARNING: using slow attention. Flash Attention requires PyTorch >= 2.0")
            # causal mask to ensure that attention is only applied to the left in the input sequence
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                        .view(1, 1, config.block_size, config.block_size))

    def forward(self, x, enhanced_feature_id=None):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        qkv = self.c_attn(x)
        if enhanced_feature_id != None:
            qkv[:, :, enhanced_feature_id[0]] = enhanced_feature_id[1]
        qkv = qkv.view(B, T, self.n_head, C // self.n_head).transpose(1, 2) # (B, nh, T, hs)

        # causal self-attention; Self-attend: (B, nh, T, hs) x (B, nh, hs, T) -> (B, nh, T, T)
        if self.flash:
            # efficient attention using Flash Attention CUDA kernels
            y = torch.nn.functional.scaled_dot_product_attention(qkv, qkv, qkv, attn_mask=None, dropout_p=self.dropout if self.training else 0, is_causal=True)
        else:
            # manual implementation of attention
            att = (qkv @ qkv.transpose(-2, -1)) * (1.0 / math.sqrt(qkv.size(-1)))
            att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ qkv # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 2).contiguous().view(B, T, C) # re-assemble all head outputs side by side

        # output projection
        y = self.resid_dropout(self.c_proj(y))
        return y

class MLP(nn.Module):
    """ISTA Block"""
    def __init__(self, config):
        super().__init__()
        self.weight = nn.Parameter(torch.Tensor(4 * config.n_embd, config.n_embd))
        with torch.no_grad():
            init.kaiming_uniform_(self.weight)
        self.relu = nn.ReLU()
        self.step_size = 0.1
        self.lambd = 0.1

    def forward(self, x, enhanced_feature_id=None):
        z_init = F.relu(self.step_size * F.linear(x, self.weight, bias=None) - self.step_size * self.lambd)

        # compute D^T * D * z_init
        x1 = F.linear(z_init, self.weight.t(), bias=None)
        grad_1 = F.linear(x1, self.weight, bias=None)

        # compute D^T * x
        grad_2 = F.linear(x, self.weight, bias=None)

        # compute negative gradient update: step_size * (D^T * x - D^T * D * x)
        grad_update = self.step_size * (grad_2 - grad_1) - self.step_size * self.lambd

        # output_sparse_code = F.relu(z_init + grad_update)
        output_sparse_code = self.relu(z_init + grad_update)
        
        # print(output_sparse_code.shape)

        if enhanced_feature_id != None:
            # print("enhancing feature of ISTA feature id: ", enhanced_feature_id)
            # print("output_sparse_code shape: ", output_sparse_code.shape)
            output_sparse_code[:, :, enhanced_feature_id[0]] = enhanced_feature_id[1]

        output = F.linear(output_sparse_code, self.weight.t(), bias=None)
        
        return output

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.prenorm_1 = LayerNorm(config.n_embd)
        self.attn = CausalSelfAttention(config)
        self.prenorm_2 = LayerNorm(config.n_embd)
        self.ista = MLP(config)

    def forward(self, x, enhanced_feature_id=None):
        if enhanced_feature_id != None:
            if enhanced_feature_id[3] == 'attn':
                x = x + self.attn(self.prenorm_1(x), enhanced_feature_id)
                x = self.ista(self.prenorm_2(x))
            else:
                x = x + self.attn(self.prenorm_1(x))
                x = self.ista(self.prenorm_2(x), enhanced_feature_id)
        else:
            x = x + self.attn(self.prenorm_1(x))
            x = self.ista(self.prenorm_2(x))
        return x

@dataclass
class CRATEConfig:
    block_size: int = 1024
    vocab_size: int = 50304 # GPT-2 vocab_size of 50257, padded up to nearest multiple of 64 for efficiency
    n_layer: int = 12
    n_head: int = 12
    n_embd: int = 768
    dropout: float = 0.0
    bias: bool = True # True: bias in Linears and LayerNorms, like GPT-2. False: a bit better and faster

class CRATE(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            drop = nn.Dropout(config.dropout),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = LayerNorm(config.n_embd, bias=True),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        # with weight tying when using torch.compile() some warnings get generated:
        # "UserWarning: functional_call was passed multiple values for tied weights.
        # This behavior is deprecated and will be an error in future versions"
        # not 100% sure what this is, so far seems to be harmless. TODO investigate
        self.transformer.wte.weight = self.lm_head.weight # https://paperswithcode.com/method/weight-tying

        # init all weights
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

        # report number of parameters
        print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))

    def get_num_params(self, non_embedding=True):
        """
        Return the number of parameters in the model.
        For non-embedding count (default), the position embeddings get subtracted.
        The token embeddings would too, except due to the parameter sharing these
        params are actually used as weights in the final layer, so we include them.
        """
        n_params = sum(p.numel() for p in self.parameters())
        if non_embedding:
            n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None, enhanced_feature_id=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device).unsqueeze(0) # shape (1, t)

        # forward the CRATE model itself
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (1, t, n_embd)
        x = self.transformer.drop(tok_emb + pos_emb)
        for i, block in enumerate(self.transformer.h):
            if enhanced_feature_id and (i == enhanced_feature_id[2]):
                x = block(x, enhanced_feature_id)
            else:
                x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            # inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(x[:, :, :]) # note: using list [-1] to preserve the time dim
            loss = None

        return logits, loss

    def crop_block_size(self, block_size):
        # model surgery to decrease the block size if necessary
        # e.g. we may load the GPT2 pretrained model checkpoint (block size 1024)
        # but want to use a smaller block size for some smaller, simpler model
        assert block_size <= self.config.block_size
        self.config.block_size = block_size
        self.transformer.wpe.weight = nn.Parameter(self.transformer.wpe.weight[:block_size])
        for block in self.transformer.h:
            if hasattr(block.attn, 'bias'):
                block.attn.bias = block.attn.bias[:,:,:block_size,:block_size]

    @classmethod
    def from_pretrained(cls, model_type, override_args=None):
        assert model_type in {'crate', 'crate-medium', 'crate-large', 'crate-xl'}
        override_args = override_args or {} # default to empty dict
        # only dropout can be overridden see more notes below
        assert all(k == 'dropout' for k in override_args)

        # n_layer, n_head and n_embd are determined from model_type
        config_args = {
            'crate-1l':      dict(n_layer=1,  n_head=1,  n_embd=768),  # 12M params
            'crate':         dict(n_layer=12, n_head=12, n_embd=768),  # 124M params
            'crate-medium':  dict(n_layer=24, n_head=16, n_embd=1024), # 350M params
            'crate-large':   dict(n_layer=36, n_head=20, n_embd=1280), # 774M params
            'crate-xl':      dict(n_layer=48, n_head=25, n_embd=1600), # 1558M params
        }[model_type]
        print("forcing vocab_size=50257, block_size=1024, bias=True")
        config_args['vocab_size'] = 50257 # always 50257 for GPT model checkpoints
        config_args['block_size'] = 1024 # always 1024 for GPT model checkpoints
        config_args['bias'] = True # always True for GPT model checkpoints
        # we can override the dropout rate, if desired
        if 'dropout' in override_args:
            print(f"overriding dropout rate to {override_args['dropout']}")
            config_args['dropout'] = override_args['dropout']
        # create a from-scratch initialized minGPT model
        config = CRATEConfig(**config_args)
        model = CRATE(config)

        return model

    def configure_optimizers(self, weight_decay, learning_rate, betas, device_type):
        # start with all of the candidate parameters
        param_dict = {pn: p for pn, p in self.named_parameters()}
        # filter out those that do not require grad
        param_dict = {pn: p for pn, p in param_dict.items() if p.requires_grad}
        # create optim groups. Any parameters that is 2D will be weight decayed, otherwise no.
        # i.e. all weight tensors in matmuls + embeddings decay, all biases and layernorms don't.
        decay_params = [p for n, p in param_dict.items() if p.dim() >= 2]
        nodecay_params = [p for n, p in param_dict.items() if p.dim() < 2]
        optim_groups = [
            {'params': decay_params, 'weight_decay': weight_decay},
            {'params': nodecay_params, 'weight_decay': 0.0}
        ]
        num_decay_params = sum(p.numel() for p in decay_params)
        num_nodecay_params = sum(p.numel() for p in nodecay_params)
        print(f"num decayed parameter tensors: {len(decay_params)}, with {num_decay_params:,} parameters")
        print(f"num non-decayed parameter tensors: {len(nodecay_params)}, with {num_nodecay_params:,} parameters")
        # Create AdamW optimizer and use the fused version if it is available
        fused_available = 'fused' in inspect.signature(torch.optim.AdamW).parameters
        use_fused = fused_available and device_type == 'cuda'
        extra_args = dict(fused=True) if use_fused else dict()
        optimizer = torch.optim.AdamW(optim_groups, lr=learning_rate, betas=betas, **extra_args)
        print(f"using fused AdamW: {use_fused}")

        return optimizer

    def estimate_mfu(self, fwdbwd_per_iter, dt):
        """ estimate model flops utilization (MFU) in units of A100 bfloat16 peak FLOPS """
        # first estimate the number of flops we do per iteration.
        # see PaLM paper Appendix B as ref: https://arxiv.org/abs/2204.02311
        N = self.get_num_params()
        cfg = self.config
        L, H, Q, T = cfg.n_layer, cfg.n_head, cfg.n_embd//cfg.n_head, cfg.block_size
        flops_per_token = 6*N + 12*L*H*Q*T
        flops_per_fwdbwd = flops_per_token * T
        flops_per_iter = flops_per_fwdbwd * fwdbwd_per_iter
        # express our flops throughput as ratio of A100 bfloat16 peak flops
        flops_achieved = flops_per_iter * (1.0/dt) # per second
        flops_promised = 312e12 # A100 GPU bfloat16 peak flops is 312 TFLOPS
        mfu = flops_achieved / flops_promised
        return mfu

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None, enhanced_feature_id=None):
        """
        Take a conditioning sequence of indices idx (LongTensor of shape (b,t)) and complete
        the sequence max_new_tokens times, feeding the predictions back into the model each time.
        Most likely you'll want to make sure to be in model.eval() mode of operation for this.
        """
        pred_logits = []
        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            # forward the model to get the logits for the index in the sequence
            logits, _ = self(idx_cond, enhanced_feature_id=enhanced_feature_id)
            pred_logits.append(logits)
            # pluck the logits at the final step and scale by desired temperature
            logits = logits[:, -1, :] / temperature
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # apply softmax to convert logits to (normalized) probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)
            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

        return idx, pred_logits

In [ ]:
import tiktoken

enc = tiktoken.get_encoding("gpt2")
encode = lambda s: enc.encode(s, allowed_special={"<|endoftext|>"})
decode = lambda l: enc.decode(l)

input_text = "My cat is cute because it has two"
input_tokens = encode(input_text)
print(input_tokens)

In [ ]:
import copy

device = "cpu"
torch.set_grad_enabled(False)
model_url = '/home/ubuntu/nanogpt4crate/out/ckpt-crate-3l-overparam.pt'

checkpoint = torch.load(model_url, map_location=device)
gptconf = CRATEConfig(**checkpoint["model_args"])
model_nanogpt = CRATE(gptconf)
state_dict = checkpoint["model"]
transformer_lens_state_dict = copy.deepcopy(state_dict)

# Some monkey business going on here. Nanogpt models saved after torch.compile() have this unwanted prefix. I need to remove it to get the expected output
# But, I also want to test my function with a state dict containing the unwanted prefix
# This is also how it's handled in nanogpt: https://github.com/karpathy/nanoGPT/blob/eba36e84649f3c6d840a93092cb779a260544d08/train.py#L171
unwanted_prefix = "_orig_mod."
for k, v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix) :]] = state_dict.pop(k)
model_nanogpt.load_state_dict(state_dict)

sample_input = torch.tensor([input_tokens])
expected_output = model_nanogpt(sample_input)[0].argmax(dim=-1)
print("Logits argmax:", expected_output)
print("Logits output:", decode(expected_output.tolist()[0]))
output_tokens = model_nanogpt.generate(sample_input, 10)[0]
print("Generation output:", decode(output_tokens.tolist()[0]))

In [ ]:
output_logits = model_nanogpt(sample_input)[0][0][-1]
highest_token_idx = torch.argsort(output_logits, descending=True)[:10].tolist()
for i in range(10):
    print(enc.decode([highest_token_idx[i]]))

In [ ]:
import einops

import transformer_lens
import transformer_lens.utils as utils
from transformer_lens.hook_points import (
    HookedRootModule,
    HookPoint,
)  # Hooking utilities
from transformer_lens import (
    HookedTransformer,
    HookedTransformerConfig,
    FactoredMatrix,
    ActivationCache,
)


When testing, set bias=False for the first, non bias model and bias = True for the second model that has bias. This converts the model to Transformers Lens format, forwards the same sample output, and asserts that the output matches the expected output.

In [ ]:
def convert_nanogpt_weights(
    old_state_dict, cfg: HookedTransformerConfig, bias: bool = False
):
    """For https://github.com/karpathy/nanoGPT
    There are two complications with converting nanogpt models:
    The first is that some state dicts have an unwanted prefix on keys that needs to be removed.
    The second is that the models can be saved with or without bias. By default, there
    is no bias. This function can handle both cases."""
    # Nanogpt models saved after torch.compile() have this unwanted prefix
    # This is a simple way to remove it
    unwanted_prefix = "_orig_mod."
    for k, v in list(old_state_dict.items()):
        if k.startswith(unwanted_prefix):
            old_state_dict[k[len(unwanted_prefix) :]] = old_state_dict.pop(k)

    new_state_dict = {}
    new_state_dict["pos_embed.W_pos"] = old_state_dict["transformer.wpe.weight"]
    new_state_dict["embed.W_E"] = old_state_dict["transformer.wte.weight"]
    # print(old_state_dict["transformer.wte.weight"].shape)

    print(old_state_dict.keys())
    new_state_dict["ln_final.w"] = old_state_dict["transformer.ln_f.weight"]
    new_state_dict["ln_final.b"] = old_state_dict["transformer.ln_f.bias"]
    new_state_dict["unembed.W_U"] = old_state_dict["lm_head.weight"].T


    for layer in range(cfg.n_layers):
        layer_key = f"transformer.h.{layer}"

        # attn block
        new_state_dict[f"blocks.{layer}.ln1.w"] = old_state_dict[
            f"{layer_key}.prenorm_1.weight"
        ]
        # A bias of zeros is required for folding layer norm
        new_state_dict[f"blocks.{layer}.ln1.b"] = torch.zeros_like(
            old_state_dict[f"{layer_key}.prenorm_1.bias"]
        )
        new_state_dict[f"blocks.{layer}.ln2.w"] = old_state_dict[
            f"{layer_key}.prenorm_2.weight"
        ]
        new_state_dict[f"blocks.{layer}.ln2.b"] = torch.zeros_like(
            old_state_dict[f"{layer_key}.prenorm_2.bias"]
        )

        W = old_state_dict[f"{layer_key}.attn.c_attn.weight"]
        # W_Q, W_K, W_V = torch.tensor_split(W, 3, dim=0)
        W_Q = einops.rearrange(W, "(i h) m->i m h", i=cfg.n_heads)
        W_K = einops.rearrange(W, "(i h) m->i m h", i=cfg.n_heads)
        W_V = einops.rearrange(W, "(i h) m->i m h", i=cfg.n_heads)
        new_state_dict[f"blocks.{layer}.attn.W_Q"] = W_Q
        new_state_dict[f"blocks.{layer}.attn.W_K"] = W_K
        new_state_dict[f"blocks.{layer}.attn.W_V"] = W_V

        W_O = old_state_dict[f"{layer_key}.attn.c_proj.weight"]
        W_O = einops.rearrange(W_O, "m (i h)->i h m", i=cfg.n_heads)
        new_state_dict[f"blocks.{layer}.attn.W_O"] = W_O
        new_state_dict[f"blocks.{layer}.attn.b_O"] = old_state_dict[
            f"{layer_key}.attn.c_proj.bias"
        ]

        # MLP block
        new_state_dict[f"blocks.{layer}.mlp.weight"] = old_state_dict[
            f"{layer_key}.ista.weight"
        ]
        # new_state_dict[f"blocks.{layer}.mlp.W_out"] = old_state_dict[
        #     f"{layer_key}.mlp.c_proj.weight"
        # ].T

        # if bias:
        new_state_dict[f"blocks.{layer}.ln1.b"] = old_state_dict[
            f"{layer_key}.prenorm_1.bias"
        ]
        new_state_dict[f"blocks.{layer}.ln2.b"] = old_state_dict[
            f"{layer_key}.prenorm_2.bias"
        ]
        # new_state_dict[f"blocks.{layer}.mlp.b_in"] = old_state_dict[
        #     f"{layer_key}.mlp.c_fc.bias"
        # ]
        # new_state_dict[f"blocks.{layer}.mlp.b_out"] = old_state_dict[
        #     f"{layer_key}.mlp.c_proj.bias"
        # ]

        # B = old_state_dict[f"{layer_key}.attn.c_attn.bias"]
        # B_Q, B_K, B_V = torch.tensor_split(B, 3, dim=0)
        # B_Q = einops.rearrange(B_Q, "(i h)->i h", i=cfg.n_heads)
        # B_K = einops.rearrange(B_K, "(i h)->i h", i=cfg.n_heads)
        # B_V = einops.rearrange(B_V, "(i h)->i h", i=cfg.n_heads)
        # new_state_dict[f"blocks.{layer}.attn.b_Q"] = B_Q
        # new_state_dict[f"blocks.{layer}.attn.b_K"] = B_K
        # new_state_dict[f"blocks.{layer}.attn.b_V"] = B_V

        # new_state_dict[f"blocks.{layer}.mlp.b_in"] = old_state_dict[
        #     f"{layer_key}.mlp.c_fc.bias"
        # ].T
        # new_state_dict[f"blocks.{layer}.mlp.b_out"] = old_state_dict[
        #     f"{layer_key}.mlp.c_proj.bias"
        # ].T

    return new_state_dict


for name, param in state_dict.items():
    # if name.startswith("transformer.h.0") or not name.startswith("transformer.h"):
    print(name, param.shape)

cfg = HookedTransformerConfig(
    n_layers = 3,
    d_model = 128,
    d_head = 32,
    n_heads = 4,
    d_mlp = 512,
    d_vocab = 50304,
    n_ctx = 1024,
    dtype=torch.float32,
    act_fn="relu",
    attention_dir='causal',
    normalization_type="LN"
)
model = HookedTransformer(cfg)
model.to('cpu')

model.load_and_process_state_dict(convert_nanogpt_weights(transformer_lens_state_dict, cfg, bias=False), fold_ln=False)
# generation = model.generate(sample_input, 10, stop_at_eos=False)
# print(generation)

logits, cache = model.run_with_cache(sample_input)
logits = logits.argmax(dim=-1)
# model_output = model(sample_input).argmax(dim=-1)
# print(model_output)
print("logits:", logits)
print("expected_output:", expected_output)
print(expected_output == logits)
# assert torch.all(expected_output.to(device) == logits), "The outputs are not equal"

## Per-layer logits output Check

In [ ]:
activation = {}

def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

In [ ]:
inspect_layer = 'checkpoint'
model_nanogpt.transformer.h[2].prenorm_1.register_forward_hook(get_activation(inspect_layer))
model_nanogpt.eval()
with torch.no_grad():
    model_nanogpt(sample_input)
print(activation[inspect_layer][0][2][-2].item())

inspect_hook = 'blocks.2.ln1.hook_normalized'
print(cache[inspect_hook][0][2][-2].item())

print(torch.all(activation[inspect_layer][0] == cache[inspect_hook][0]))

In [ ]:
activation[inspect_layer].shape, cache[inspect_hook].shape

In [ ]:
torch.all(model.blocks[0].mlp.weight == model_nanogpt.transformer.h[0].ista.weight)

In [ ]:
cache